# NLPTK: тулкит для Natural Language Processing

## Работа с классом Text: основные методы и свойства

In [1]:
# минимально необходимые внешние модули
#!pip install chardet
#!pip install dawg
#!pip install gensim
#!pip install pattern
#!pip install razdel
#!pip install pymorphy2

import sys, os
# сменим рабочую директорию на ту, где расположен каталог с библиотекой nlptk
os.chdir(r'D:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')

In [2]:
#from nlptk.mining.utils import datapath

In [2]:
# добавим этот путь в sys.path, чтобы интерпретатор увидел нашу библиотеку
sys.path.append(r'D:\INSTALL\Python3\PROJECTS\REPOS\nlp_toolkit')


In [3]:
import sys
from pprint import pprint
import nlptk
from nlptk.mining.text import *



In [6]:
# создаем экземпляр класса Prep c настройками по умолчанию
prep = Prep()
# это то, что будет использоваться при обработке текста
prep.__dict__

{'sentencizer': <function nlptk.misc.mixins.SentencizerMixin.sentencize_nltk(text, *args, lang='english', **kwargs)>,
 'tokenizer': functools.partial(<function TokenizerMixin.toktok_tokenize at 0x0E0989C0>, strip='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'),
 'tagger': <function nlptk.misc.mixins.TaggerMixin.tagger_nltk(tokens, *args, lang='eng', **kwargs)>,
 'lemmatizer': functools.partial(<function LemmatizerMixin.lemmatize_nltk at 0x0E098780>, pos=True, normalize_uppercase=<method 'capitalize' of 'str' objects>)}

In [7]:
# набор правил для очистки текста; применяются на этапе работы с необработанным текстом;
rules_clean = OrderedDict(
        roman_numerals=(True,) # здесь еще много всяких правил используемых по умолчанию
)
# набор правил для фильтра токенов; применяются на этапе запроса слов\токенов\лемм из тэгированного текста
# могут быть включены на уровне составления вокабуляра класса Corpus 
# (по умолчанию фильтрация выключена, поэтому вокабуляр максимально полный)
rules_filter = OrderedDict(
        by_tagpos=(True,{},{"FW","POS"}), # фильтр по тегам частей речи - игнорировать иностранные слова и possessive ending parent's
        punctuation=True, # по умолчанию
        short=(True,3),   # по умолчанию
        stopwords=True,   # по умолчанию
        ifnotin_lexicon=True
        #trailing_chars=True
)

# классы которым передаются правила, сами же экземпляры классов передаются в конструктор класса Text
clean = TextCleaner(rules_clean)
filters = TokenFilter(rules_filter)   
    
        

In [20]:
clean.rules

OrderedDict([('hyphenation', (True,)),
             ('accent', (True,)),
             ('tags', (True,)),
             ('urls', (True,)),
             ('numeric', (True,)),
             ('nonletter_sequences', (True,)),
             ('quotes', (True,)),
             ('multiple_whitespaces', (True,)),
             ('roman_numerals', (True,))])

In [8]:
filters.rules

In [8]:
# класс Text может принять в качестве ввода непосредственно текст (нужно указать это в параметре input='text', 
# по дефолту input="filename")
intake='''THE PICTURE OF DORIAN GRAY

BY

OSCAR WILDE. The artist is the creator of beautiful things. PREFACE'''
text1 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text1)

THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde
The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing
PREFACE⁄NN⁄Preface


In [9]:
text1.summarize(3)

[(0.5, 'THE PICTURE OF DORIAN GRAY BY OSCAR WILDE'),
 (0.5, 'The artist is the creator of beautiful things')]

In [8]:
text1.sents()

[TaggedSentence(
 	'THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde',
 	 n=0
 ), TaggedSentence(
 	'The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 	 n=1
 ), TaggedSentence(
 	'PREFACE⁄NN⁄Preface',
 	 n=2
 )]

In [9]:
text1.sents(1).tokens()

(Token(word='The', idx=0, pos='DT', lemma='The', nsent=1),
 Token(word='artist', idx=1, pos='NN', lemma='artist', nsent=1),
 Token(word='is', idx=2, pos='VBZ', lemma='be', nsent=1),
 Token(word='the', idx=3, pos='DT', lemma='the', nsent=1),
 Token(word='creator', idx=4, pos='NN', lemma='creator', nsent=1),
 Token(word='of', idx=5, pos='IN', lemma='of', nsent=1),
 Token(word='beautiful', idx=6, pos='JJ', lemma='beautiful', nsent=1),
 Token(word='things', idx=7, pos='NNS', lemma='thing', nsent=1))

In [10]:
text1.sents(1).words()

('the', 'artist', 'is', 'the', 'creator', 'of', 'beautiful', 'things')

In [11]:
text1.sents(1).words(uniq=True)

('creator', 'things', 'beautiful', 'the', 'artist', 'is', 'of')

In [18]:
text1.sents(1).text

'The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing'

In [19]:
text1.sents(1).raw

'The artist is the creator of beautiful things'

In [12]:
text1.words(lower=False, filtrate=False)

['THE',
 'PICTURE',
 'OF',
 'DORIAN',
 'GRAY',
 'BY',
 'OSCAR',
 'WILDE',
 'The',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things',
 'PREFACE']

In [13]:
text1.words(lower=True, filtrate=False)

['the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things',
 'preface']

In [14]:
text1.words(lower=True, filtrate=False, uniq=True)

['picture',
 'wilde',
 'things',
 'of',
 'is',
 'by',
 'creator',
 'the',
 'artist',
 'preface',
 'dorian',
 'beautiful',
 'gray',
 'oscar']

In [15]:
text1.sents(0).untagging()

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde')]

In [20]:
text1.count("words")

17

In [21]:
text1.count("words",uniq=True)

16

In [14]:
text1.count("lemmas")

17

In [15]:
text1.count("lemmas",uniq=True)

16

In [21]:
text1.count("lemmas",uniq=True,pos='JJ')

2

In [10]:
text1.postags(pos='JJ')


FreqDist({'dorian': 1, 'beautiful': 1})

In [12]:
text1.keywords().topn(7)

['oscar wilde',
 'dorian gray',
 'beautiful things',
 'picture',
 'preface',
 'artist',
 'creator']

In [10]:
intake = 'The Picture of Dorian Gray By Oscar Wilde'
text2 = Text(intake, prep, clean, filters, input='text', inplace=True)
print(text2)

The⁄DT⁄The Picture⁄NN⁄Picture of⁄IN⁄of Dorian⁄JJ⁄Dorian Gray⁄NNP⁄Gray By⁄IN⁄By Oscar⁄NNP⁄Oscar Wilde⁄NNP⁄Wilde


In [46]:
text2.sents(0).untagging()

[('The', 'DT', 'The'),
 ('Picture', 'NN', 'Picture'),
 ('of', 'IN', 'of'),
 ('Dorian', 'JJ', 'Dorian'),
 ('Gray', 'NNP', 'Gray'),
 ('By', 'IN', 'By'),
 ('Oscar', 'NNP', 'Oscar'),
 ('Wilde', 'NNP', 'Wilde')]

In [11]:
print(repr(text2))

Text(
	name='StringIO',
	nsents=1,
	nwords=8,
	nlemmas=8
)


## А теперь создадим корпус из коллекции документов

In [12]:
# MODULEDIR это путь до директории каталога nlptk относительно которого мы получаем путь к коллекции документов
source = os.path.abspath(os.path.join(nlptk.MODULEDIR,r'corpus\en'))
corpus = Corpus(Path(source,"*.txt"), prep, clean, filters)
corpus.verbose = True  # при True будет выводить небольшую отладочную информацию об этапах обработки каждого документа
#corpus.rewrite = True # перетегировать все тексты, даже если они сохранены 
#corpus.filtrate = True # применить фильтры при формировании общего вокабуляра; по умолчанию False
print(help(corpus.tfidf))


Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, token=None, texts: List[List[str]] = [], sort=False, top=0) method of nlptk.mining.text.Corpus instance

None


In [13]:
texts = []

for text in corpus:
    # можно выводить объекты текстов text и что-то с ними делать;  по умолчанию объекты  Text генераторы предложений,
    # но класс Corpus передает им параметр inplace=True, что заставляет  их отработать на месте 
    texts.append(text) # в списке texts будут уже отработанные экзмепляры, а не генераторы
    #for i,sent in enumerate(text):
    #    sent # можно выводить предложения при обработке, но это замедлит работу 
        
print('Done!')


reading:   ..\mining\data\Austin Pride and Prejudice.txt
elapsed:   3.1801819801330566
vocab: ['pride', 'and', 'prejudice', 'jane', 'austen', 'chapter', 'it', 'is', 'a',...]
add vocab: 4.085233688354492
reading:   ..\mining\data\bronte_jane_txt.txt
elapsed:   8.901509046554565
vocab: ['jane', 'eyre', 'by', 'charlotte', 'bronte', 'preface', 'a', 'preface',...]
add vocab: 10.415595769882202
reading:   ..\mining\data\bronte_wuthering_txt.txt
elapsed:   4.319247007369995
vocab: ['wuthering', 'heights', 'emily', 'bronte', 'chapter', 'i', 'i', 'have',...]
add vocab: 5.397308588027954
reading:   ..\mining\data\doyle_the_adventures.txt
elapsed:   3.574204444885254
vocab: ['the', 'adventures', 'of', 'sherlock', 'holmes', 'by', 'sir', 'arthur',...]
add vocab: 4.5972630977630615
reading:   ..\mining\data\dreiser_sister.txt
elapsed:   7.708440780639648
vocab: ['sister', 'carrie', 'by', 'theodore', 'dreiser', 'chapter', 'i', 'the',...]
add vocab: 9.20452618598938
reading:   ..\mining\data\Edgar All

In [17]:
print(repr(corpus))

Corpus(
	names=[],
	ndocs=0,
	nwords=0,
	nlemmas=0,
	nhapaxes=0
)


In [9]:
# Последнее предложение в корпусе, которое осталось в этой переменной после цикла обработки
# сами предложения в объекте корпуса никак не сохраняются в целях экономии памяти. 
# Сохраняются только всевозможные частотные словари
sent

TaggedSentence(
	'THE⁄DT⁄The END⁄NN⁄End',
	 n=6503
)

In [16]:
# работаем с последним текстом, который интерпретатор сохранил в этой переменной
text

Text(
	name='wilde_picture_of_dorian_gray_txt',
	nsents=6506,
	nwords=79794,
	nlemmas=5797
)

In [49]:
# объект text при печати на консоль отображается как текст аналогичный сохраненному на диске тегированному тексту
pprint(str(text).split("\n")[:10])

['THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray '
 'BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde THE⁄DT⁄The PREFACE⁄NNP⁄Preface '
 'The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of '
 'beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 'To⁄TO⁄To reveal⁄VB⁄reveal art⁄NN⁄art and⁄CC⁄and conceal⁄VB⁄conceal '
 'the⁄DT⁄the artist⁄NN⁄artist is⁄VBZ⁄be art⁄JJ⁄art s⁄NN⁄s aim⁄NN⁄aim',
 'The⁄DT⁄The critic⁄NN⁄critic is⁄VBZ⁄be he⁄PRP⁄he who⁄WP⁄who can⁄MD⁄can '
 'translate⁄VB⁄translate into⁄IN⁄into another⁄DT⁄another manner⁄NN⁄manner '
 'or⁄CC⁄or a⁄DT⁄a new⁄JJ⁄new material⁄NN⁄material his⁄PRP$⁄his '
 'impression⁄NN⁄impression of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
 'The⁄DT⁄The highest⁄JJS⁄high as⁄IN⁄a the⁄DT⁄the lowest⁄JJS⁄low form⁄NN⁄form '
 'of⁄IN⁄of criticism⁄NN⁄criticism is⁄VBZ⁄be a⁄DT⁄a mode⁄NN⁄mode of⁄IN⁄of '
 'autobiography⁄NN⁄autobiography',
 'Those⁄DT⁄Those who⁄WP⁄who find⁄VBP⁄find ugly⁄JJ⁄ugly meanings⁄NNS⁄meaning '
 'in⁄IN⁄in beautiful⁄JJ

In [8]:
# получаем общее числу слов в тексте, но после фильтрации
len(text.words(filtrate=True))

34925

In [9]:
len(text.words(filtrate=False))

79794

In [10]:
# число слов в тексте без фильтрации
text.nwords

79794

In [11]:
# считаем сколько всего слов есть в корпусе по всем документам
sum(corpus.ccfs().values())

1927580

In [ ]:
# ПРОВЕРЯЕМ ЧТО ПОДСЧЕТЫ ВХОЖДЕНИЙ СЛОВ ОДИНАКОВЫ КАК ДЛЯ ОБЪЕКТОВ ТЕКСТОВ, ТАК И НА УРОВНЕ ВОКАБУЛЯРА КОРПУСА

In [17]:
[(txt.filename,txt.nwords) for txt in texts]

[('Austin Pride and Prejudice.txt', 122447),
 ('bronte_jane_txt.txt', 186971),
 ('bronte_wuthering_txt.txt', 118794),
 ('doyle_the_adventures.txt', 105136),
 ('dreiser_sister.txt', 156304),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 2335),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 2419),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 2126),
 ('fitzgerald_great_gatsby_txt.txt', 48484),
 ('Franz Kafka - Metamorphosis.txt', 22335),
 ('John Steinbeck - Of Mice and Men.txt', 31025),
 ('kipling_jungle_book_txt.txt', 51737),
 ('london_white_txt.txt', 72830),
 ('stevenson_treasure_island_txt.txt', 69728),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 316418),
 ('stoker_dracula_txt.txt', 161394),
 ('twain_tom_sawyer_txt.txt', 72935),
 ('walter_scott_ivanhoe_txt.txt', 194098),
 ('wells_invisible_man_txt.txt', 49693),
 ('wells_war_of_the_worlds_txt.txt', 60577),
 ('wilde_picture_of_dorian_gray_txt.txt', 79794)]

In [18]:
# считаем число слов для каждого текст по сумме частот из каждого словаря cfs
# это должно совпасть с тем числом, которое каждый Text хранит в свойстве nwords
[(name, sum(corpus.cfs(n).values())) for n,name in enumerate(corpus.filenames())]

[('Austin Pride and Prejudice.txt', 122447),
 ('bronte_jane_txt.txt', 186971),
 ('bronte_wuthering_txt.txt', 118794),
 ('doyle_the_adventures.txt', 105136),
 ('dreiser_sister.txt', 156304),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 2335),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 2419),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 2126),
 ('fitzgerald_great_gatsby_txt.txt', 48484),
 ('Franz Kafka - Metamorphosis.txt', 22335),
 ('John Steinbeck - Of Mice and Men.txt', 31025),
 ('kipling_jungle_book_txt.txt', 51737),
 ('london_white_txt.txt', 72830),
 ('stevenson_treasure_island_txt.txt', 69728),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 316418),
 ('stoker_dracula_txt.txt', 161394),
 ('twain_tom_sawyer_txt.txt', 72935),
 ('walter_scott_ivanhoe_txt.txt', 194098),
 ('wells_invisible_man_txt.txt', 49693),
 ('wells_war_of_the_worlds_txt.txt', 60577),
 ('wilde_picture_of_dorian_gray_txt.txt', 79794)]

In [16]:
# объект предложения в виде экземпляра класса TaggedSentence; n это порядковый номер предложения
text.sents(0)

TaggedSentence(
	'THE⁄DT⁄The PICTURE⁄NN⁄Picture OF⁄IN⁄Of DORIAN⁄JJ⁄Dorian GRAY⁄NNP⁄Gray BY⁄IN⁄By OSCAR⁄NNP⁄Oscar WILDE⁄NNP⁄Wilde THE⁄DT⁄The PREFACE⁄NNP⁄Preface The⁄DT⁄The artist⁄NN⁄artist is⁄VBZ⁄be the⁄DT⁄the creator⁄NN⁄creator of⁄IN⁄of beautiful⁄JJ⁄beautiful things⁄NNS⁄thing',
	 n=0
)

In [22]:
# легко догадаться, что это число слов в предложении
print(text.sents(0).nwords)
print(len(text.sents(0).tokens()))
print(len(text.sents(0).words()))
print(len(text.sents(0).words(uniq=True)))
print(len(text.sents(0).lemmas(uniq=True)))

18
18
18
14
14


In [44]:
# а так мы получаем все предложения из текста не длиннее одного слова
text.sents(max_words=1)[:3]

[TaggedSentence(
 	'Hallward⁄NN⁄Hallward',
 	 n=177
 ), TaggedSentence(
 	'Gray⁄NN⁄Gray',
 	 n=212
 ), TaggedSentence(
 	'Harry⁄NNP⁄Harry',
 	 n=236
 )]

In [43]:
# Давайте усложним условия: не менее трех, но не более пяти слов
text.sents(min_words=3,max_words=5)[:3]

[TaggedSentence(
 	'This⁄DT⁄This is⁄VBZ⁄be a⁄DT⁄a fault⁄NN⁄fault',
 	 n=5
 ), TaggedSentence(
 	'For⁄IN⁄For these⁄DT⁄these there⁄EX⁄there is⁄VBZ⁄be hope⁄NN⁄hope',
 	 n=7
 ), TaggedSentence(
 	'That⁄DT⁄That is⁄VBZ⁄be all⁄DT⁄all',
 	 n=11
 )]

In [48]:
# А если мы хотим получить предложение в более информативном виде? Класс Token к вашим услугам.
# Метод tokens трансформирует предложение в список экземпляров этого класса.
text.sents(0).tokens(lower=True)

(Token(token='the', idx=0, pos='DT', lemma='the', nsent=0),
 Token(token='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(token='of', idx=2, pos='IN', lemma='of', nsent=0),
 Token(token='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(token='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(token='by', idx=5, pos='IN', lemma='by', nsent=0),
 Token(token='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(token='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(token='the', idx=8, pos='DT', lemma='the', nsent=0),
 Token(token='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(token='the', idx=10, pos='DT', lemma='the', nsent=0),
 Token(token='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(token='is', idx=12, pos='VBZ', lemma='be', nsent=0),
 Token(token='the', idx=13, pos='DT', lemma='the', nsent=0),
 Token(token='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(token='of', idx=15, pos='IN', lemma='of', n

In [49]:
# Попробуем включить фильтрацию (это те самые rules_filter, которые мы передавали в объект Corpus, 
# но которые по умолчанию отключены параметром corpus.filtrate=False, 
# чтобы в объекте корпуса сохранялась статистика по всем словам
text.sents(0).tokens(lower=True,filtrate=True)

(Token(token='picture', idx=1, pos='NN', lemma='picture', nsent=0),
 Token(token='dorian', idx=3, pos='JJ', lemma='dorian', nsent=0),
 Token(token='gray', idx=4, pos='NNP', lemma='gray', nsent=0),
 Token(token='oscar', idx=6, pos='NNP', lemma='oscar', nsent=0),
 Token(token='wilde', idx=7, pos='NNP', lemma='wilde', nsent=0),
 Token(token='preface', idx=9, pos='NNP', lemma='preface', nsent=0),
 Token(token='artist', idx=11, pos='NN', lemma='artist', nsent=0),
 Token(token='creator', idx=14, pos='NN', lemma='creator', nsent=0),
 Token(token='beautiful', idx=16, pos='JJ', lemma='beautiful', nsent=0),
 Token(token='things', idx=17, pos='NNS', lemma='thing', nsent=0))

In [17]:
# все слова предложения с указанным номером
text.sents(0).words()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'is',
 'the',
 'creator',
 'of',
 'beautiful',
 'things')

In [18]:
# все леммы предложения с указанным номером
text.sents(0).lemmas()

('the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface',
 'the',
 'artist',
 'be',
 'the',
 'creator',
 'of',
 'beautiful',
 'thing')

In [8]:
# разтегированное представаление предложения
text.sents(0).untagging()

[('THE', 'DT', 'The'),
 ('PICTURE', 'NN', 'Picture'),
 ('OF', 'IN', 'Of'),
 ('DORIAN', 'JJ', 'Dorian'),
 ('GRAY', 'NNP', 'Gray'),
 ('BY', 'IN', 'By'),
 ('OSCAR', 'NNP', 'Oscar'),
 ('WILDE', 'NNP', 'Wilde'),
 ('THE', 'DT', 'The'),
 ('PREFACE', 'NNP', 'Preface'),
 ('The', 'DT', 'The'),
 ('artist', 'NN', 'artist'),
 ('is', 'VBZ', 'be'),
 ('the', 'DT', 'the'),
 ('creator', 'NN', 'creator'),
 ('of', 'IN', 'of'),
 ('beautiful', 'JJ', 'beautiful'),
 ('things', 'NNS', 'thing')]

In [16]:
# слова предложения отфильтрованные по частям речи - сущ. (в ед. и мн. числе
text.sents(0).words(pos={"NN","NNS"})

('picture', 'artist', 'creator', 'things')

In [22]:
# леммы предложения отфильтованные по частям речи - сущ. (в ед. и мн. числе)
text.sents(0).lemmas(pos={"NN","NNS"})


('picture', 'artist', 'creator', 'thing')

In [23]:
# все леммы чей postag начинается на N, то есть любые существительные
text.sents(0).lemmas(pos="N")


('picture', 'gray', 'oscar', 'wilde', 'preface', 'artist', 'creator', 'thing')

In [19]:
text.sents(0).lemmas(pos="JJ")
# имя dorian неверно определяется как прилагательное, но здесь играет роль неоднозначность слова, так как такое прилагательное 
# тоже существует в английском

('dorian', 'beautiful')

In [24]:
# Предложение как есть, но уже без пунктуации, так как она была удалена на этапе очистки текста фильтрами TextCleaner
text.sents(0).raw

'THE PICTURE OF DORIAN GRAY BY OSCAR WILDE THE PREFACE The artist is the creator of beautiful things'

In [13]:
text.words(filtrate=False)[:10]

['the',
 'picture',
 'of',
 'dorian',
 'gray',
 'by',
 'oscar',
 'wilde',
 'the',
 'preface']

In [13]:
# Можно получить частотные словари по любой части речи
verbs = text.postags("VERB")[:10]
verbs

FreqDist({'be': 3543, 'have': 1579, 'say': 388, 'do': 370, 'go': 294, 'know': 260, 'come': 230, 'look': 207, 'make': 205, 'think': 192, ...})

In [15]:
#verbs.hapaxes()

In [22]:
adjectives = text.postags("ADJ")[:10]
adjectives

FreqDist({'dorian': 303, 't': 162, 'good': 138, 'own': 130, 'great': 82, 'little': 82, 'young': 78, 'other': 72, 'more': 65, 'such': 65, ...})

In [16]:
possessive = text.postags("POS")[:10]
possessive

FreqDist({'s': 2})

In [24]:
# генерация skipgram из текста (имплементация из nltk)
for n,s in enumerate(text.skipgrams(3,2)):
    print(s)
    if n > 10:
        break

('picture', 'dorian', 'gray')
('picture', 'dorian', 'oscar')
('picture', 'dorian', 'wilde')
('picture', 'gray', 'oscar')
('picture', 'gray', 'wilde')
('picture', 'oscar', 'wilde')
('dorian', 'gray', 'oscar')
('dorian', 'gray', 'wilde')
('dorian', 'gray', 'preface')
('dorian', 'oscar', 'wilde')
('dorian', 'oscar', 'preface')
('dorian', 'wilde', 'preface')


In [25]:
# генерация ngram из текста (имплементация из nltk)
for n,s in enumerate(text.ngrams(3)):
    print(s)
    if n > 10:
        break

('picture', 'dorian', 'gray')
('dorian', 'gray', 'oscar')
('gray', 'oscar', 'wilde')
('oscar', 'wilde', 'preface')
('wilde', 'preface', 'artist')
('preface', 'artist', 'creator')
('artist', 'creator', 'beautiful')
('creator', 'beautiful', 'thing')
('beautiful', 'thing', 'reveal')
('thing', 'reveal', 'art')
('reveal', 'art', 'conceal')
('art', 'conceal', 'artist')


In [42]:
#[(n,corpus.cfs(n).get('said',0)) for n in range(corpus.ndocs)]
[(n,corpus.cfs(n,'said')) for n in range(corpus.ndocs)]

[(0, 401),
 (1, 583),
 (2, 375),
 (3, 486),
 (4, 1474),
 (5, 24),
 (6, 0),
 (7, 2),
 (8, 236),
 (9, 51),
 (10, 390),
 (11, 430),
 (12, 70),
 (13, 341),
 (14, 1289),
 (15, 567),
 (16, 356),
 (17, 1451),
 (18, 531),
 (19, 166),
 (20, 262)]

In [41]:
[(n,corpus.tfidf(n,'said')) for n in range(corpus.ndocs)]


[(0, 0.00015978223910706062),
 (1, 0.00015213410481186327),
 (2, 0.0001540171352386233),
 (3, 0.0002255366362268298),
 (4, 0.00046010787942562473),
 (5, 0.0005014834861097941),
 (6, 0),
 (7, 4.589855519231613e-05),
 (8, 0.00023749028017461358),
 (9, 0.00011140803101146338),
 (10, 0.000613317132186253),
 (11, 0.0004055080617905132),
 (12, 4.689429482164278e-05),
 (13, 0.00023860495040408915),
 (14, 0.00019875772432161856),
 (15, 0.00017140676285405884),
 (16, 0.00023814764440005225),
 (17, 0.0003647360004216731),
 (18, 0.0005213526487426482),
 (19, 0.00013370036898700366),
 (20, 0.00016020030343623828)]

In [9]:
# набор наиболее весомых по метрике TFIDF слов в каждом тексте
res = [(n,
        corpus.tfidf(n,sort=-1)[0:7],
        name
       ) 
     for n,name in enumerate(corpus.filenames())
]
# берем текст по индексу 3 - Приключения Шерлока Холмса
res[3]

(3,
 [('holmes', 0.013378570292080937),
  ('sherlock', 0.002169412950320131),
  ('watson', 0.0014991888798649405),
  ('lestrade', 0.0011004018854958346),
  ('rucastle', 0.0011004018854958346),
  ('mccarthy', 0.001071443941140681),
  ('simon', 0.0008946032784825284)],
 'doyle_the_adventures.txt')

In [29]:
# отображение слов из 3-го текста в порядковый номер в словаре tdidf
ordered_by_tfidf = {word:n for n,(word,_) in enumerate(corpus.tfidf(3,sort=-1))}

In [36]:
#ordered_by_tfidf

In [39]:
# как видим, хотя метрика и дает наибольшие веса словам важным для контекста произведения - именам главных персонажей,
# но очень быстро скатывается до уровня частоупотребляемых вместе с этими словами сокращений типа mr.
# чья документтная частота очень и очень высока 
# (но их можно удалять фильтрацией, которую мы на уровне корпуса не включали, поэтому все стоп-слова и остались на месте)

# документная частота слова
corpus.dfs('mr')

16

In [40]:
# 
ordered_by_tfidf['mr'] # на  8 месте если упоядочить слова по TFIDF 

8

In [32]:
ordered_by_tfidf['said']

83

In [33]:
# число вхождений в текст по индексу 3
corpus.cfs(3,'mr')

275

In [34]:
# документная частота слова "holmes"
corpus.dfs('holmes')

1

In [24]:
# число вхождений в текст по индексу 3
print(corpus.cfs(3,'holmes'))   # число вхождений словоформы
print(texts[3].vocab['holmes']) # число вхождений лемм
print(texts[3].words(filtrate=False).count('holmes'))
print(texts[3].lemmas(filtrate=False).count('holmes'))

462
462
462
462


In [11]:
print(corpus.cfs(3,'investigation'))       # число вхождений словоформы
print(corpus.cfs(3,'investigations'))
print(texts[3].vocab.get('investigation')) # число вхождений леммы == числу вхождений словоформ, которые образуют данную лемму
print(texts[3].words(filtrate=False).count('investigation'))  # число вхождений словоформы
print(texts[3].lemmas(filtrate=False).count('investigation')) # число вхождений леммы

12
3
15
12
15


In [30]:
corpus.tfidf(6,"red")

0.00020712636290766274

In [30]:
corpus.cfs(3,'said')

486

In [31]:
corpus.dfs('said')

20

In [32]:
corpus.tfidf(3,'said')

0.00022664684706076194

In [26]:
corpus.tfidf(6,sort=-1)[:10]

[('prospero', 0.007551523202290425),
 ('courtiers', 0.0038881773578561016),
 ('waltzers', 0.0037757616011452125),
 ('mummer', 0.0037757616011452125),
 ('prince', 0.0036805490437174567),
 ('ebony', 0.0029662764061375003),
 ('musicians', 0.002413282532933419),
 ('revellers', 0.002413282532933419),
 ('suite', 0.002071538600240377),
 ('sable', 0.0020565044356389405)]

In [8]:
# тоже самое (только в виде упорядоченного списка) - с использованием параметра top
corpus.tfidf(6,top=10)

['prospero',
 'courtiers',
 'waltzers',
 'mummer',
 'prince',
 'ebony',
 'musicians',
 'revellers',
 'suite',
 'sable']

In [46]:
help(corpus.tfidf)

Help on method tfidf in module nlptk.mining.text:

tfidf(n_doc, texts: List[List[str]] = None, token=None, sort=False) method of nlptk.mining.text.Corpus instance



In [31]:
print(corpus.cfs(1,'rochester'))
print(sum(corpus.cfs(1).values()))
print(corpus.dfs('rochester'))
print(corpus.tf(1,'rochester'))
print(corpus.idf('rochester'))
print(corpus.tfidf(1,token='rochester'))


312
185598
1
0.0016810525975495426
3.044522437723423
0.005118002352232825


In [32]:
input("--cfc--")
pprint(corpus.cfs(0,sort=-1)[:10])
input("--ccfc--")
pprint(corpus.ccfs(sort=-1)[:10])
input("--dfc--")
pprint(corpus.dfs(sort=-1)[:10])
input("--tfidf--")

#for n in range(corpus.ndocs):
#    pprint(corpus.tfidf(n,sort=-1)[:10]) 
            

--cfc--
[('the', 4320),
 ('to', 4127),
 ('of', 3596),
 ('and', 3529),
 ('her', 2216),
 ('i', 2046),
 ('a', 1945),
 ('in', 1861),
 ('was', 1843),
 ('she', 1703)]
--ccfc--
[('the', 104005),
 ('and', 64431),
 ('of', 53145),
 ('to', 51249),
 ('a', 42386),
 ('i', 34454),
 ('in', 29525),
 ('he', 28953),
 ('was', 26894),
 ('it', 24706)]
--dfc--
[('over', 21),
 ('once', 21),
 ('moment', 21),
 ('a', 21),
 ('minutes', 21),
 ('by', 21),
 ('four', 21),
 ('not', 21),
 ('from', 21),
 ('loud', 21)]
--tfidf--


''

In [20]:
# общее число слов-одиночек
all_hapaxes = corpus.hapaxes()
len(all_hapaxes)

18047

In [9]:
# н-да, вот что значит не включать фильтрацию на корпусе ...
sorted(all_hapaxes)[:10]

['a-bleeding',
 'a-blowing',
 'a-buttin',
 'a-callin',
 'a-chaffin',
 'a-changing',
 'a-crossin',
 'a-doin',
 'a-doing',
 'a-done']

In [21]:
# получаем слова-одиночки для каждого текста в корпусе
hapaxes = [(path, corpus.hapaxes(n)) for n,path in enumerate(corpus.filenames())]
hapaxes[0][1][:10]

['austen',
 'rightful',
 'morris',
 'grown-up',
 'newcomers',
 'over-scrupulous',
 'vexing',
 'sarcastic',
 'develop',
 'solace']

In [22]:
# сколько у нас гапаксов на каждый текст
[(h[0],len(h[1])) for h in hapaxes]

[('Austin Pride and Prejudice.txt', 2472),
 ('bronte_jane_txt.txt', 5873),
 ('bronte_wuthering_txt.txt', 4254),
 ('doyle_the_adventures.txt', 3758),
 ('dreiser_sister.txt', 4547),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 545),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 585),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 383),
 ('fitzgerald_great_gatsby_txt.txt', 3222),
 ('Franz Kafka - Metamorphosis.txt', 1291),
 ('John Steinbeck - Of Mice and Men.txt', 1371),
 ('kipling_jungle_book_txt.txt', 2212),
 ('london_white_txt.txt', 3190),
 ('stevenson_treasure_island_txt.txt', 2951),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 8813),
 ('stoker_dracula_txt.txt', 4471),
 ('twain_tom_sawyer_txt.txt', 3691),
 ('walter_scott_ivanhoe_txt.txt', 5737),
 ('wells_invisible_man_txt.txt', 2915),
 ('wells_war_of_the_worlds_txt.txt', 3330),
 ('wilde_picture_of_dorian_gray_txt.txt', 3542)]

In [31]:
# гапаксы с вычислением по всем словоформам, результат должен быть идентичен коду выше 
[(txt.filename, len(txt.hapaxes(words=True))) for txt in texts]

[('Austin Pride and Prejudice.txt', 2472),
 ('bronte_jane_txt.txt', 5873),
 ('bronte_wuthering_txt.txt', 4254),
 ('doyle_the_adventures.txt', 3758),
 ('dreiser_sister.txt', 4547),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 545),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 585),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 383),
 ('fitzgerald_great_gatsby_txt.txt', 3222),
 ('Franz Kafka - Metamorphosis.txt', 1291),
 ('John Steinbeck - Of Mice and Men.txt', 1371),
 ('kipling_jungle_book_txt.txt', 2212),
 ('london_white_txt.txt', 3190),
 ('stevenson_treasure_island_txt.txt', 2951),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 8813),
 ('stoker_dracula_txt.txt', 4471),
 ('twain_tom_sawyer_txt.txt', 3691),
 ('walter_scott_ivanhoe_txt.txt', 5737),
 ('wells_invisible_man_txt.txt', 2915),
 ('wells_war_of_the_worlds_txt.txt', 3330),
 ('wilde_picture_of_dorian_gray_txt.txt', 3542)]

In [32]:
# а так гапаксы будут считаться по леммам и, поскольку словарь лемм для каждого текста уже имеется в экземпляре каждого текста, 
# вычисляется гораздо быстрее
[(txt.filename, len(txt.hapaxes())) for txt in texts]

[('Austin Pride and Prejudice.txt', 1835),
 ('bronte_jane_txt.txt', 4646),
 ('bronte_wuthering_txt.txt', 3157),
 ('doyle_the_adventures.txt', 2963),
 ('dreiser_sister.txt', 3620),
 ('Edgar Allan Poe The Cask of Amontillado.txt', 486),
 ('Edgar Allan Poe The Masque of the Red Death.txt', 496),
 ('Edgar Allan Poe The Tell-Tale Heart.txt', 338),
 ('fitzgerald_great_gatsby_txt.txt', 2594),
 ('Franz Kafka - Metamorphosis.txt', 996),
 ('John Steinbeck - Of Mice and Men.txt', 1073),
 ('kipling_jungle_book_txt.txt', 1698),
 ('london_white_txt.txt', 2507),
 ('stevenson_treasure_island_txt.txt', 2245),
 ('Stivenson_Fall-or-Dodge-in-Hell_RuLit_Me.txt', 7309),
 ('stoker_dracula_txt.txt', 3527),
 ('twain_tom_sawyer_txt.txt', 2862),
 ('walter_scott_ivanhoe_txt.txt', 4445),
 ('wells_invisible_man_txt.txt', 2316),
 ('wells_war_of_the_worlds_txt.txt', 2674),
 ('wilde_picture_of_dorian_gray_txt.txt', 2806)]

In [23]:
ndoc = 3
print(len(texts[ndoc].words(filtrate=False)))

hp = texts[ndoc].hapaxes(words=True)
print(len(hp))
print(sorted(hp)[-110:len(hp)])
'ycuea' in hp

105136
3758
['wedged', 'wedlock', 'wee', 'weedy', 'weekly', 'weigh', 'weighing', 'weird', 'welcome', 'welcomed', 'well-cut', 'well-groomed', 'well-lit', 'well-nigh', 'well-nurtured', 'well-opened', 'well-remembered', 'wellington', 'westbury', 'western', 'westphail', 'westward', 'wheal', 'wheel', 'wheeled', 'wherever', 'whims', 'whine', 'whined', 'whirling', 'whishing', 'whisper', 'whispering', 'white-aproned', 'white-counterpaned', 'whiten', 'whiter', 'whither', 'whittington', 'wholesome', 'whoso', 'wicker-work', 'wicket', 'widow', 'wig', 'wight', 'wigmore', 'wigs', 'wilderness', 'wilful', 'wilhelm', 'will-o', 'willingly', 'willows', 'wilton', 'wimpole', 'win', 'winced', 'wincing', 'wind-swept', 'windfall', 'window-sill', 'windowsill', 'wine', 'wine-cellar', 'wines', 'winking', 'winter', 'wintry', 'wiry', 'wisdom', 'wishing', 'withdrawn', 'witnesses', 'wits', 'wives', 'woke', 'womanhood', 'wondered', 'wooded', 'wooden-leg', 'wooden-legged', 'worker', 'workmen', 'world-wide', 'worlds', 

True

In [24]:
print(sum(corpus.cfs(ndoc).values()))
ln = len(hapaxes[ndoc][1])
print(ln)
print(sorted(hapaxes[ndoc][1])[-110:ln])

105136
3758
['wedged', 'wedlock', 'wee', 'weedy', 'weekly', 'weigh', 'weighing', 'weird', 'welcome', 'welcomed', 'well-cut', 'well-groomed', 'well-lit', 'well-nigh', 'well-nurtured', 'well-opened', 'well-remembered', 'wellington', 'westbury', 'western', 'westphail', 'westward', 'wheal', 'wheel', 'wheeled', 'wherever', 'whims', 'whine', 'whined', 'whirling', 'whishing', 'whisper', 'whispering', 'white-aproned', 'white-counterpaned', 'whiten', 'whiter', 'whither', 'whittington', 'wholesome', 'whoso', 'wicker-work', 'wicket', 'widow', 'wig', 'wight', 'wigmore', 'wigs', 'wilderness', 'wilful', 'wilhelm', 'will-o', 'willingly', 'willows', 'wilton', 'wimpole', 'win', 'winced', 'wincing', 'wind-swept', 'windfall', 'window-sill', 'windowsill', 'wine', 'wine-cellar', 'wines', 'winking', 'winter', 'wintry', 'wiry', 'wisdom', 'wishing', 'withdrawn', 'witnesses', 'wits', 'wives', 'woke', 'womanhood', 'wondered', 'wooded', 'wooden-leg', 'wooden-legged', 'worker', 'workmen', 'world-wide', 'worlds', 

In [16]:
texts[3].vocab['conan']

1

In [30]:
hapaxes[3][1][:10]

['ycuea',
 'aiaeeeneii',
 'ia',
 'eieae',
 'aðoaea',
 'walsall',
 'manifested',
 'mauritius',
 'solely',
 'survived']

In [29]:
corpus.cfs(3,'manifested')

1

In [28]:
# общекорпусная частота слова
corpus.ccfs('manifested')

28

In [17]:

print(corpus.cfs(0,'have'))
print(texts[0].words(filtrate=False).count('have'))

842
842


In [16]:
texts[0]

Text(
	name='Austin Pride and Prejudice',
	encoding='utf-8',
	nsents=5950,
	nwords=122447,
	nlemmas=4950
)

In [15]:
sum(corpus.cfs(0).values())


122447

In [20]:
corpus.ccfs("am")

2083

In [13]:
# частота слова для текста корпуса с номером 0
corpus.cfs(0,'vicious')

1

In [31]:
corpus.cfs(9,"gregor")

298

In [32]:
corpus.ccfs("s") # остатки притяжательных окончаний, которые токенизатор отделяет от основы

7905

In [14]:
corpus.dfs('s')

21

In [24]:
# тегов притяжательных окончаний нет, так как nltk.tag_pos умеет их определять только в виде "'s", а всю пунктуацию 
# в начале и в конце токена стрипает токенайзер (точнее, обертка вокруг него), поэтому в тексте остаются только токены вида "s"
pos,_ = texts[10].postags("POS")
pprint(pos)


FreqDist({})


In [16]:
import nltk
print(nltk.pos_tag(["Gregor", "'s"]))
#[('Gregor', 'NNP'), ("'s", 'POS')]

print(nltk.pos_tag(["Gregor", "s"]))
#[('Gregor', 'NNP'), ('s', 'NN')]

# токены в которых притяжательное окончание присутствует tag_pos не умеет определять как NNP, то есть имена собственные
print(nltk.pos_tag(["Gregor's"]))
#[("Gregor's", 'NN')]


[('Gregor', 'NNP'), ("'s", 'POS')]
[('Gregor', 'NNP'), ('s', 'NN')]
[("Gregor's", 'NN')]


In [23]:
# выводим слова с сортировкой по возрастанию частот
corpus.cfs(9,sort=1)[:10]

[('troubled', 1),
 ('transformed', 1),
 ('vermin', 1),
 ('armour-like', 1),
 ('domed', 1),
 ('divided', 1),
 ('arches', 1),
 ('sections', 1),
 ('bedding', 1),
 ('cover', 1)]

In [36]:
texts[12].keywords(rating=('rake', dict(max_words=3))).topn(10)

['something might happen',
 'far from it',
 'i m busy',
 'bring a light',
 'it was delightful',
 'i just counted',
 'night had fallen',
 'something was impending',
 'don t know',
 'white fang paused']

In [28]:
# много памяти для текстов имеющих больше 20 тыс. словоупотреблений
print(texts[6].nwords) # рассказ Эдгара По 'Маска Красной Смерти'
texts[6].summarize(10, scores=False)

2419


['And now was acknowledged the presence of the Red Death',
 'Blood was its Avatar and its seal the redness and the horror of blood',
 'It was in this apartment also that there stood against the western wall a gigantic clock of ebony',
 'There were much of the beautiful much of the wanton much of the bizarre something of the terrible and not a little of that which might have excited disgust',
 'To and fro in the seven chambers there stalked in fact a multitude of dreams',
 'The figure was tall and gaunt and shrouded from head to foot in the habiliments of the grave',
 'And then for a moment all is still and all is silent save the voice of the clock',
 'But in spite of these things it was a gay and magnificent revel',
 'It was in the blue room where stood the prince with a group of pale courtiers by his side',
 'But these other apartments were densely crowded and in them beat feverishly the heart of life']

In [13]:
ndoc = 7
words =  [set(sent.lemmas(uniq=True)) for sent in texts[ndoc].sents()]


In [64]:
len(words)

6506

In [33]:
import itertools

def similarity(s1, s2):
        '''Мера сходства - коэффициент Сёренсена - 
        https://ru.wikipedia.org/wiki/Коэффициент_Сёренсена
        отношение количества одинаковых слов в 
        предложениях к суммарной длине предложений.
        ''' 
        if not len(s1) or not len(s2):
            return 0.0
        
        return len(s1.intersection(s2))/(1.0 * (len(s1) + len(s2)))


In [65]:
nsents = texts[ndoc].nsents    
pairs = itertools.combinations(range(nsents), 2)


In [66]:
scores = []
for i, j in pairs:
    sim = similarity(words[i], words[j])
    if sim:
        scores.append((i, j, sim)) 
    

print(len(scores))
print(scores[:10])

11378564
[(0, 1, 0.125), (0, 2, 0.15151515151515152), (0, 3, 0.125), (0, 4, 0.11538461538461539), (0, 5, 0.05555555555555555), (0, 6, 0.16666666666666666), (0, 7, 0.05263157894736842), (0, 8, 0.16), (0, 9, 0.08), (0, 10, 0.05)]
